# Data Wrangling Using MongoDb

In [1]:
OSM_FILE = 'syracuse_new_york.osm'

In [2]:
import xml.etree.ElementTree as ET
import pprint

#Iterative parcing to create a dictionary of all the tags available with in the OSM XML file. 
#parse through it with ElementTree and count the number of unique element types to get an overview of the dataset.

def count_tags(filename):
    """count tags in filename.
    
    add to dict if the key not exist, increment otherwise."""
    
    tag_count = {} # creating dictionary for tags existing in filename  
    for _, element in ET.iterparse(filename, events=("start",)):
        add_tag(element.tag, tag_count)
    return tag_count

def add_tag(tag, tag_count):
    if tag in tag_count:
        tag_count[tag] += 1 # initialize 1 if tag not exist and increment otherwise 
    else:
        tag_count[tag] = 1

tags = count_tags(OSM_FILE)
pprint.pprint(tags)


{'bounds': 1,
 'member': 5556,
 'nd': 328273,
 'node': 279462,
 'osm': 1,
 'relation': 795,
 'tag': 215643,
 'way': 35500}


In [3]:
import re

# check the "k" value for each three tag categories in a dictionary
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):

    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map(OSM_FILE)
pprint.pprint(keys)

{'lower': 99834, 'lower_colon': 108213, 'other': 7595, 'problemchars': 1}


In [4]:
# number of  users that have contributed to the map of Syracuse (New York).

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        try:
            users.add(element.attrib['uid'])
        except KeyError:
            pass
        element.clear() # clear memory
    return users

users = process_map(OSM_FILE)
len(users)

248

### Auding (Identification of  potential problems in the dataset) 

#### Street type issues

In [5]:
from collections import defaultdict

#create dictionary of street types, a string to audit, a regex to match against that string, 
# and the list of expected street types

street_type_regex = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Avenue", "Boulevard", "Commons", "Court", "Drive", "Lane", "Parkway", 
                         "Place", "Road", "Square", "Street", "Trail"]

mapping = {'Ave'  : 'Avenue',
           'Blvd' : 'Boulevard',
           'Dr'   : 'Drive',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'Rd'   : 'Road',
           'St.'   : 'Street',
           'Rd.'   : 'Road',
           'St'   : 'Street',
           'street' :"Street",
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'N'   : "North",
           'S'   : "South",
           'E'   : "East",
           'W'   : "West",
           'Sq'   : "Square"}

In [6]:
def audit_street_type(street_types, street_name):
    m = street_type_regex.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types

strt_types_ = audit(OSM_FILE)
pprint.pprint(dict(strt_types_))


{'11': set(['Route 11', 'US Route 11']),
 '298': set(['State Route 298']),
 '31': set(['New York 31', 'State Highway 31', 'State Route 31']),
 'Center': set(['Carousel Center']),
 'Circle': set(["Children's Circle",
                'Flat Car Circle',
                'Sherbrooke Circle',
                'Walsh Circle']),
 'Courts': set(['Presidential Courts']),
 'East': set(['Erie Boulevard East', 'Lawrence Road East']),
 'Path': set(['Bronco Path']),
 'Plaza': set(['Presidential Plaza', 'Shop City Plaza']),
 'Rowe': set(['Basile Rowe']),
 'Run': set(['Trotters Ridge Run']),
 'St': set(['James St']),
 'Terrace': set(['Clark Terrace',
                 'Drumlins Terrace',
                 'Erregger Terrace',
                 'Onondaga Terrace',
                 'Standish Terrace',
                 'Sycamore Terrace']),
 'Turnpike': set(['East Seneca Turnpike', 'West Seneca Turnpike']),
 'Way': set(['Baptist Way']),
 'West': set(['Bear Street West',
              'Hiawatha Boulevard West',

#### Adjusting street type problems

In [7]:
def update_name(name, mapping, regex):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)

    return name

for street_type, ways in strt_types_.iteritems():
    for name in ways:
        better_name = update_name(name, mapping, street_type_regex)
        print name, "=>", better_name

Route 11 => Route 11
US Route 11 => US Route 11
Trotters Ridge Run => Trotters Ridge Run
Carousel Center => Carousel Center
Richmond Road West => Richmond Road West
Bear Street West => Bear Street West
Hiawatha Boulevard West => Hiawatha Boulevard West
New York 31 => New York 31
State Route 31 => State Route 31
State Highway 31 => State Highway 31
Shop City Plaza => Shop City Plaza
Presidential Plaza => Presidential Plaza
James St => James Street
Standish Terrace => Standish Terrace
Drumlins Terrace => Drumlins Terrace
Clark Terrace => Clark Terrace
Sycamore Terrace => Sycamore Terrace
Erregger Terrace => Erregger Terrace
Onondaga Terrace => Onondaga Terrace
Baptist Way => Baptist Way
West Seneca Turnpike => West Seneca Turnpike
East Seneca Turnpike => East Seneca Turnpike
Bronco Path => Bronco Path
Children's Circle => Children's Circle
Walsh Circle => Walsh Circle
Flat Car Circle => Flat Car Circle
Sherbrooke Circle => Sherbrooke Circle
Erie Boulevard East => Erie Boulevard East
Lawr

#### Zipcode issues

In [8]:
from collections import defaultdict

def audit_zipcode(invalid_zipcodes, zipcode):
    zip_cod = re.match(r'^\d{5}$', zipcode)# matches postcodes which are of 5 digits format
    if not zip_cod:
        invalid_zipcodes[zip_cod].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

zipcode_ = audit_zip(OSM_FILE)
pprint.pprint(dict(zipcode_))

{None: set(['13202-1107',
            '13204-1243',
            '132059211',
            '13206-2238',
            '13210-1053',
            '13210-1203',
            '13214-1303',
            '132179211',
            '13218-1185',
            '13219-331',
            '13224-1110'])}


#### Zipcode adjustments 

In [9]:
def update_zip(zipcode):
               
    if re.findall('(\d{5})-\d{4}', zipcode):
                return re.sub('(\d{5})-\d{4}', '\\1', zipcode)# filter out the first group of digits before a hypen
    if re.findall('(\d{5})\d{4}', zipcode):
                return re.sub('(\d{5})\d{4}', '\\1', zipcode)# filter out the first 5 digits out of the 9 digits 
    
    else:
        return (re.findall(r'\d+', zipcode))[0]

for street_type, ways in zipcode_.iteritems():
    for name in ways:
        better_name = update_zip(name)
        print name, "=>", better_name

132059211 => 13205
13206-2238 => 13206
13218-1185 => 13218
13202-1107 => 13202
13210-1203 => 13210
13210-1053 => 13210
13224-1110 => 13224
132179211 => 13217
13219-331 => 13219
13214-1303 => 13214
13204-1243 => 13204


# Data structure alteration to Json file format

In [10]:
import re
import codecs
import json
addr_regex = re.compile(r'^addr\:')
strt_regex = re.compile(r'^street')

from datetime import datetime

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

# all attributes of "node" and "way" should be turned into regular key/value pairs
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :  #2 types of top level tags: "node" and "way"
        node['type'] = element.tag
        address = {}
       
        for item in element.attrib:
            if item in CREATED:# attributes in the CREATED array are added under a key "created"
                if 'created' not in node:
                    node['created'] = {}
                node['created'][item] = element.get(item)
            elif item in ['lat', 'lon']:
                continue
            else:
                node[item] = element.get(item)
       
    # attributes for latitude and longitude are added to a "pos" array
        if 'lat' in element.attrib and 'lon' in element.attrib:
            node['pos'] = [float(element.get('lat')), float(element.get('lon'))]


        

# second level tag "k" value starts with "addr:", added to a dictionary "address"
# second level tag "k" values do not start with "addr:", but contains ":",converted to an updated key

        # Parcing through second level tags
        for tag in element.iter('tag'):
            key   = tag.attrib['k']
            value = tag.attrib['v']
            if not problemchars.search(key):# problemchars are ignored

                # second level tag "k" value beginning with addr and have a colon ("addr:")
                if lower_colon.search(key) and key.find('addr') == 0:
                    if 'address' not in node:
                        node['address'] = {}
                    s_tag = key.split(':')[1]
                    if s_tag == 'street':
                        # updating attributes         
                        node['address'][s_tag] = update_name(tag.attrib['v'], mapping, street_type_regex)
                    elif s_tag == 'postcode': 
                        node['address'][s_tag] = update_zip(tag.attrib['v'])
                    else:    
                        node['address'][s_tag] = value

                # tags that don't start with "addr"
                elif not key.find('addr') == 0:
                    if key not in node:
                        node[key] = value
                else:
                    node["tag:" + key] = value

        # parce through nodes
        for nd in element.iter('nd'):
            if 'node_refs' not in node:
                node['node_refs'] = []
            node['node_refs'].append(nd.attrib['ref'])

        return node
    else:
        return None




def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data
process_map(OSM_FILE)

[{'created': {'changeset': '12299360',
   'timestamp': '2012-07-18T21:35:48Z',
   'uid': '722137',
   'user': 'OSMF Redaction Account',
   'version': '4'},
  'id': '18654197',
  'pos': [43.2166491, -76.0489984],
  'type': 'node'},
 {'created': {'changeset': '12299360',
   'timestamp': '2012-07-18T21:35:48Z',
   'uid': '722137',
   'user': 'OSMF Redaction Account',
   'version': '3'},
  'id': '18654198',
  'pos': [43.2168121, -76.04978],
  'type': 'node'},
 {'created': {'changeset': '12299360',
   'timestamp': '2012-07-18T21:35:48Z',
   'uid': '722137',
   'user': 'OSMF Redaction Account',
   'version': '3'},
  'id': '18654199',
  'pos': [43.2170624, -76.0502226],
  'type': 'node'},
 {'created': {'changeset': '12299360',
   'timestamp': '2012-07-18T21:35:48Z',
   'uid': '722137',
   'user': 'OSMF Redaction Account',
   'version': '3'},
  'id': '18654201',
  'pos': [43.2179959, -76.0513695],
  'type': 'node'},
 {'created': {'changeset': '12299360',
   'timestamp': '2012-07-18T21:35:48Z',

In [11]:
data = process_map(OSM_FILE)

In [12]:
from pymongo import MongoClient

In [13]:
client  = MongoClient('mongodb://localhost:27017')
db = client.examples

In [22]:
[db.Syracuse_osm_NY3.insert(e) for e in data]

C:\Users\Barneiy\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':


[ObjectId('57900f36c1a2311dec8e1595'),
 ObjectId('57900f36c1a2311dec8e1596'),
 ObjectId('57900f36c1a2311dec8e1597'),
 ObjectId('57900f36c1a2311dec8e1598'),
 ObjectId('57900f36c1a2311dec8e1599'),
 ObjectId('57900f36c1a2311dec8e159a'),
 ObjectId('57900f36c1a2311dec8e159b'),
 ObjectId('57900f36c1a2311dec8e159c'),
 ObjectId('57900f36c1a2311dec8e159d'),
 ObjectId('57900f36c1a2311dec8e159e'),
 ObjectId('57900f36c1a2311dec8e159f'),
 ObjectId('57900f36c1a2311dec8e15a0'),
 ObjectId('57900f36c1a2311dec8e15a1'),
 ObjectId('57900f36c1a2311dec8e15a2'),
 ObjectId('57900f36c1a2311dec8e15a3'),
 ObjectId('57900f36c1a2311dec8e15a4'),
 ObjectId('57900f36c1a2311dec8e15a5'),
 ObjectId('57900f36c1a2311dec8e15a6'),
 ObjectId('57900f36c1a2311dec8e15a7'),
 ObjectId('57900f36c1a2311dec8e15a8'),
 ObjectId('57900f36c1a2311dec8e15a9'),
 ObjectId('57900f36c1a2311dec8e15aa'),
 ObjectId('57900f36c1a2311dec8e15ab'),
 ObjectId('57900f36c1a2311dec8e15ac'),
 ObjectId('57900f36c1a2311dec8e15ad'),
 ObjectId('57900f36c1a231

# Syracuse data analysis in MongoDb

##### File size

In [23]:
import os
print 'The original OSM file is {} MB'.format(os.path.getsize(OSM_FILE)/1.0e6) # bytes to megabytes
print 'The JSON file is {} MB'.format(os.path.getsize(OSM_FILE + ".json")/1.0e6) # bytes to megabytes

The original OSM file is 64.200921 MB
The JSON file is 72.088946 MB


##### Number of Documents 

In [31]:
db.Syracuse_osm_NY3.find().count()

314962

##### Number of Nodes 

In [32]:
db.Syracuse_osm_NY3.find({'type':'node'}).count()

279462

##### Number of Ways

In [33]:
db.Syracuse_osm_NY3.find({'type':'way'}).count()

35500

##### Number of Users

In [34]:
len(db.Syracuse_osm_NY3.distinct('created.user'))

234

##### List of top 10 Postcodes in Syracuse

In [24]:
postcodes = db.Syracuse_osm_NY3.aggregate( [ 
    { "$match" : { "address.postcode" : { "$exists" : 1} } }, 
    { "$group" : { "_id" : "$address.postcode", "count" : { "$sum" : 1} } },  
    { "$sort" : { "count" : -1}},
      {"$limit":10}] )
print(list(postcodes))

[{u'count': 822, u'_id': u'13224'}, {u'count': 514, u'_id': u'13214'}, {u'count': 475, u'_id': u'13210'}, {u'count': 290, u'_id': u'13206'}, {u'count': 284, u'_id': u'13205'}, {u'count': 173, u'_id': u'13108'}, {u'count': 137, u'_id': u'13212'}, {u'count': 114, u'_id': u'13057'}, {u'count': 112, u'_id': u'13031'}, {u'count': 110, u'_id': u'13066'}]


##### List of top 5 Streets in Syracuse

In [25]:
street = db.Syracuse_osm_NY3.aggregate( [ 
    { "$match" : { "address.street" : { "$exists" : 1} } }, 
    { "$group" : { "_id" : "$address.street", "count" : { "$sum" : 1} } },  
    { "$sort" : { "count" : -1}},
      {"$limit":5}] )
print(list(street))

[{u'count': 265, u'_id': u'Erie Boulevard East'}, {u'count': 206, u'_id': u'Westmoreland Avenue'}, {u'count': 199, u'_id': u'South Salina Street'}, {u'count': 164, u'_id': u'East Genesee Street'}, {u'count': 155, u'_id': u'Westcott Street'}]


##### List of top 10 Cities in Syracuse

In [35]:
city = db.Syracuse_osm_NY3.aggregate( [ 
    { "$match" : { "address.city" : { "$exists" : 1} } }, 
    { "$group" : { "_id" : "$address.city", "count" : { "$sum" : 1} } },  
    { "$sort" : { "count" : -1}},
      {"$limit":10}] )
print(list(city))

[{u'count': 2866, u'_id': u'Syracuse'}, {u'count': 781, u'_id': u'DeWitt'}, {u'count': 211, u'_id': u'Marcellus'}, {u'count': 158, u'_id': u'Camillus'}, {u'count': 116, u'_id': u'Fayetteville'}, {u'count': 105, u'_id': u'North Syracuse'}, {u'count': 104, u'_id': u'Manlius'}, {u'count': 94, u'_id': u'East Syracuse'}, {u'count': 69, u'_id': u'Liverpool'}, {u'count': 35, u'_id': u'Solvay'}]


##### List of 10 amenities in Syracuse

In [28]:
amenity = db.Syracuse_osm_NY3.aggregate([{'$match': {'amenity': {'$exists': 1}}}, \
                                {'$group': {'_id': '$amenity', \
                                            'count': {'$sum': 1}}}, \
                                {'$sort': {'count': -1}}, \
                                {'$limit': 10}])
print(list(amenity))

[{u'count': 939, u'_id': u'parking'}, {u'count': 186, u'_id': u'school'}, {u'count': 158, u'_id': u'fast_food'}, {u'count': 153, u'_id': u'restaurant'}, {u'count': 153, u'_id': u'bench'}, {u'count': 132, u'_id': u'place_of_worship'}, {u'count': 122, u'_id': u'fuel'}, {u'count': 64, u'_id': u'bank'}, {u'count': 57, u'_id': u'post_box'}, {u'count': 51, u'_id': u'pharmacy'}]


##### Top 5 Cuisines in Syracuse

In [29]:
cuisine = db.Syracuse_osm_NY3.aggregate([{"$match":{"amenity":{"$exists":1},
                                 "amenity":"restaurant",}},      
                      {"$group":{"_id":{"Food":"$cuisine"},
                                 "count":{"$sum":1}}},
                      {"$project":{"_id":0,
                                  "Food":"$_id.Food",
                                  "Count":"$count"}},
                      {"$sort":{"Count":-1}}, 
                      {"$limit":5}])
print(list(cuisine))

[{u'Food': None, u'Count': 65}, {u'Food': u'pizza', u'Count': 16}, {u'Food': u'american', u'Count': 11}, {u'Food': u'chinese', u'Count': 10}, {u'Food': u'italian', u'Count': 8}]


##### List of 5 Buildings types in Syracuse

In [30]:
building = db.Syracuse_osm_NY3.aggregate([
       {'$match': {'building': { '$exists': 1}}}, 
        {'$group': {'_id': '$building',
                    'count': {'$sum': 1}}}, 
        {'$sort': {'count': -1}},
        {'$limit': 5}])
print(list(building))

[{u'count': 8269, u'_id': u'yes'}, {u'count': 3588, u'_id': u'house'}, {u'count': 233, u'_id': u'apartments'}, {u'count': 185, u'_id': u'retail'}, {u'count': 155, u'_id': u'roof'}]
